## Problem 1

In [2]:
def read_input(filename):
    f = open(f'../inputs/{filename}.txt', 'r')

    codes = []
    while True:
        line = f.readline()
        if line == '':
            break
        codes.append(line.strip())

    f.close()
    return codes

In [3]:
num_positions = {
    'A': [3, 2],
    '0': [3, 1],
    '1': [2, 0],
    '2': [2, 1],
    '3': [2, 2],
    '4': [1, 0],
    '5': [1, 1],
    '6': [1, 2],
    '7': [0, 0],
    '8': [0, 1],
    '9': [0, 2]
}

dir_positions = {
    'A': [0, 2],
    '<': [1, 0],
    'v': [1, 1],
    '>': [1, 2],
    '^': [0, 1]
}

In [114]:
from enum import Enum

class Keypad(str, Enum):
    NUM = 'num',
    DIR = 'dir'

def paths_on_keypad(button1, button2, keypad):
    start = num_positions[button1] if keypad == Keypad.NUM else dir_positions[button1]
    end = num_positions[button2] if keypad == Keypad.NUM else dir_positions[button2]
    
    vertical_move = 'v' if start[0] < end[0] else '^'
    horizontal_move = '<' if end[1] < start[1] else '>'

    paths = []
    if (keypad == Keypad.NUM and (start[0] != 3 or end[1] != 0)) or (keypad == Keypad.DIR and (start[0] != 0 or end[1] != 0)):
        moves = []
        # start with horizontal
        for i in range(abs(start[1]-end[1])):
            moves.append(horizontal_move)
        for i in range(abs(start[0]-end[0])):
            moves.append(vertical_move)
        moves.append('A')
        paths.append(''.join(moves))
    if start[0] != end[0] and start[1] != end[1] and \
       ((keypad == Keypad.NUM and (start[1] != 0 or end[0] != 3)) or (keypad == Keypad.DIR and (start[1] != 0 or end[0] != 0))):
        moves = []
        # start with vertical
        for i in range(abs(start[0]-end[0])):
            moves.append(vertical_move)
        for i in range(abs(start[1]-end[1])):
            moves.append(horizontal_move)
        moves.append('A')
        paths.append(''.join(moves))

    return paths

In [118]:
import math

def possible_paths(code, keypad):
    prev_pos = 'A'

    paths = []
    for ch in code:
        paths_to_ch = paths_on_keypad(prev_pos, ch, keypad)
        prev_pos = ch
        paths.append(paths_to_ch)
    return paths

def find_shortest_directions_between_buttons(button1, button2, lvl):
    if lvl == 0:
        return len(paths_on_keypad(button1, button2, Keypad.DIR)[0])
    shortest = math.inf
    for path in paths_on_keypad(button1, button2, Keypad.DIR):
        prev = 'A'
        length = 0
        for ch in path:
            length += find_shortest_directions_between_buttons(prev, ch, lvl-1)
            prev = ch
        if length < shortest:
            shortest = length

    return shortest

def find_shortest_directions_for_path(path, lvl):
    prev = 'A'
    length = 0
    for ch in path:
        length += find_shortest_directions_between_buttons(prev, ch, lvl)
        prev = ch

    return length

In [112]:
def code_complexity(code):
    shortest_seq = 0
    paths = possible_paths(code, Keypad.NUM)
    for step in paths:
        shortest_for_step = math.inf
        for path in step:
            length = find_shortest_directions_for_path(path, 1)
            if length < shortest_for_step:
                shortest_for_step = length
        shortest_seq += shortest_for_step

    return shortest_seq*int(code[:-1])

In [103]:
from functools import reduce

def solve1(input_filename):
    codes = read_input(input_filename)
    return reduce(lambda x, y: x + code_complexity(y), codes, 0)

## Problem 2

In [134]:
import math

def find_shortest_directions_between_buttons2(button1, button2, lvl, memo):
    if (button1, button2, lvl) in memo:
        return memo[(button1, button2, lvl)]
    if lvl == 0:
        res = len(paths_on_keypad(button1, button2, Keypad.DIR)[0])
        memo[(button1, button2, lvl)] = res
        return res
    shortest = math.inf
    for path in paths_on_keypad(button1, button2, Keypad.DIR):
        prev = 'A'
        length = 0
        for ch in path:
            length += find_shortest_directions_between_buttons2(prev, ch, lvl-1, memo)
            prev = ch
        if length < shortest:
            shortest = length

    memo[(button1, button2, lvl)] = shortest
    return shortest

def find_shortest_directions_for_path2(path, lvl, memo):
    prev = 'A'
    length = 0
    for ch in path:
        length += find_shortest_directions_between_buttons2(prev, ch, lvl, memo)
        prev = ch

    return length

In [129]:
def code_complexity2(code, lvl, memo):
    shortest_seq = 0
    paths = possible_paths(code, Keypad.NUM)
    for step in paths:
        shortest_for_step = math.inf
        for path in step:
            length = find_shortest_directions_for_path2(path, lvl-1, memo)
            if length < shortest_for_step:
                shortest_for_step = length
        shortest_seq += shortest_for_step

    return shortest_seq*int(code[:-1])

In [130]:
def solve2(input_filename, lvl):
    codes = read_input(input_filename)

    res = 0
    memo = {}
    for code in codes:
        res += code_complexity2(code, lvl, memo)

    return res